# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [1]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).

In [2]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [3]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?

print(movies.columns)
print(ratings.columns)

print(ratings["user_id"].nunique())
print(movies["movie_id"].nunique())

print(movies.head())

Index(['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
       'genre_unknown', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')
Index(['user_id', 'movie_id', 'rating', 'unix_timestamp'], dtype='object')
943
1682
   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            imdb_url  genre_unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...              0       0   
1  http://u

1. ... How many unique user ids are present?<br>
943 unique user ids.<br>
How many unique movies are there?<br>
1,682 unique movies.

In [4]:
# Merge movies and ratings dataframes

# Merge on movie_id, and keep only the user_id, title, and ratings features.
merged_df = movies.merge(ratings, on="movie_id")[["user_id", "title", "rating"]]
merged_df.head()


,user_id,title,rating
0,308,Toy Story (1995),4
1,287,Toy Story (1995),5
2,148,Toy Story (1995),4
3,280,Toy Story (1995),4
4,66,Toy Story (1995),3


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [5]:
# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df):

  #print("Inside content_based_recommendation(user_id, df)...")

  # 1.A. Get the user's rated movies
  users_movies_df = merged_df[merged_df["user_id"] == user_id]
  #print("1.A.")
  #print(users_movies_df)


  # 1.B. Create a TF-IDF matrix using movie genres

  # Calculate the TF_IDF as by taking the movie genre columns and dividing them by the sum.
  # Set the index of the index of the resulting DataFrame to be the movie titles
  tfidf_df = (movies[genre_cols]/movies[genre_cols].sum()).set_index(movies['title'])

  #print()
  #print("1.B.")
  #print(tfidf_df)


  # 1.C. Compute the cosine similarity between movie genres
  #print()
  #print("1.C")
  cosine_similarity_array = cosine_similarity(tfidf_df)
  #print('cosine', cosine_similarity_array)

  # 1. D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.
  #print()
  # Get the subset of movies the user has seen and create a profile.
  user_movies = tfidf_df.loc[users_movies_df["title"]]
  #print(user_movies)
  user_profile = user_movies.mean()
  #print(user_profile)

  # Get movies similar to the user profile using cosine similarity.
  user_prof_similarities = cosine_similarity(user_profile.values.reshape(1, -1), tfidf_df)
  #print(user_prof_similarities)

  # Wrap the user similarities in a pandas DataFrame
  user_prof_similarities_df = pd.DataFrame(user_prof_similarities.T, index=tfidf_df.index, columns=["similarity_score"])

  # Sort descending
  sorted_user_prof_similarities_df = user_prof_similarities_df.sort_values(by="similarity_score", ascending=False)

  # Keep only the top 5
  top_5_user_similar_movies_df = sorted_user_prof_similarities_df.head()
  #print(top_5_user_similar_movies_df)

  # 1.E. Remove duplicates and movies already rated by the user.
  #print()
  # Remove duplicates
  clean_top_5_user_similar_movies_df = top_5_user_similar_movies_df[~top_5_user_similar_movies_df.index.duplicated(keep="first")]
  #print(clean_top_5_user_similar_movies_df)

  # Remove movies already rated by the user.
  clean_top_user_similar_movies_df = clean_top_5_user_similar_movies_df[~clean_top_5_user_similar_movies_df.index.isin(users_movies_df["title"])]
  #print(clean_top_user_similar_movies_df)

  # Return clean top user similar movies as recommendations in a dataframe to be further parsed or formatted.
  return clean_top_user_similar_movies_df


In [8]:
  # 1.A. Get the user's rated movies
  users_movies_df = merged_df[merged_df["user_id"] == user_id]

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [9]:
# Collaborative Filtering using User-Item Interactions
def collaborative_filtering_recommendation(user_id, df):
  # Create the user-item matrix

  # A. Create the user-item matrix using Pandas' pivot_table.
  #print("A. Create the user-item matrix using Pandas' pivot_table.")
  merged_df_pivot = merged_df.pivot_table(index="user_id", columns="title", values="rating")
  #print(merged_df_pivot)

  # B. Fill missing values with 0 (indicating no rating)
  #print("B. Fill missing values with zeros in this matrix.")

  avg_ratings = merged_df_pivot.mean(axis=1)
  merged_df_pivot = merged_df_pivot.sub(avg_ratings, axis=0)
  #print(merged_df_pivot)
  merged_df_pivot.fillna(0,inplace=True)

  # C. Calculate user-user similarity matrix using cosine similarity
  #print("C. Calculate user-user similarity matrix using cosine similarity")
  similarities = cosine_similarity(merged_df_pivot)

  # Wrap in a DataFrame.
  users_cosine_similarity_df = pd.DataFrame(similarities, index=merged_df_pivot.index, columns=merged_df_pivot.index)
  #print(users_cosine_similarity_df.head())

  # Get the similarity scores of the target user with all other users
  # D. Get the array of similarity scores of the target user with all other users from the similarity matrix.
  #print("D. Get the array of similarity scores of the target user with all other users from the similarity matrix.")

  # Get the target user's similarity values
  user_similarity_series = users_cosine_similarity_df.loc[user_id]
  # Order the values descending
  ordered_similarities = user_similarity_series.sort_values(ascending=False)
  nearest_neighbors = ordered_similarities.index
  #print(nearest_neighbors)

  # E. Find the top N most similar users (excluding the target user)
  # E. Extract, say the the top 5 most similar users (excluding the target user).
  #print("E. Extract, say the the top 5 most similar users (excluding the target user).")
  top_5_similar_users = nearest_neighbors[1:6]
  #print(top_5_similar_users)

  # F. Generate movie recommendations based on the most similar users
  #print("F. Generate movie recommendations based on the most similar users")

  # Extract the ratings of the neighbors
  neighbor_ratings = merged_df_pivot.reindex(top_5_similar_users )
  #print(neighbor_ratings)

  # Calculate the average of the nieghbors ratings of the films.
  neighbor_average_ratings_df = neighbor_ratings.mean().sort_values(ascending=False)
  #print(neighbor_average_ratings_df)

  # G.Remove duplicates from recommendations

  # Remove duplicates
  dedup_neighbor_average_ratings_df = neighbor_average_ratings_df[~neighbor_average_ratings_df.index.duplicated(keep="first")]
  #print(dedup_neighbor_average_ratings_df)

  # Instructions were less than clear so decided to check for movies already rated by user:
  # Remove movies already rated by the user.
  clean_recommended_movies_df = dedup_neighbor_average_ratings_df[~dedup_neighbor_average_ratings_df.index.isin(users_movies_df["title"])]
  #print(clean_recommended_movies_df)

  # Also decided to reduce the list of recommendations to 5.
  top_5_user_recommended_movies = clean_recommended_movies_df.iloc[0:5]
  #print(clean_recommended_movies_df.iloc[0:5])

  # Rename column 0 to similarity score (there is probably a cleaner way to do this).
  top_5_user_recommended_movies = top_5_user_recommended_movies.reset_index().rename(columns={0:"similarity_score"}).set_index("title")
  #print(top_5_user_recommended_movies)

  return top_5_user_recommended_movies


Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

In [10]:
# Test the recommendation engines

# Call the function
user_id = 308
print("User ID:", user_id)
print(content_based_recommendation(user_id, merged_df))
print(collaborative_filtering_recommendation(user_id, merged_df))

print()
user_id = 287
print("User ID:", user_id)
print(content_based_recommendation(user_id, merged_df))
print(collaborative_filtering_recommendation(user_id, merged_df))

print()
user_id = 148
print("User ID:", user_id)
print(content_based_recommendation(user_id, merged_df))
print(collaborative_filtering_recommendation(user_id, merged_df))


User ID: 308
                          similarity_score
title                                     
Starship Troopers (1997)          0.494245
                                    similarity_score
title                                               
Princess Bride, The (1987)                  0.747614
Postino, Il (1994)                          0.547614
People vs. Larry Flynt, The (1996)          0.437207
City of Lost Children, The (1995)           0.430522
Persuasion (1995)                           0.339337

User ID: 287
                                 similarity_score
title                                            
Empire Strikes Back, The (1980)          0.668134
Escape from L.A. (1996)                  0.663432
Abyss, The (1989)                        0.663432
Escape from New York (1981)              0.663432
                                    similarity_score
title                                               
Contact (1997)                              0.619162
Princess Bride

Are the recommendations similar? Do the recommendations make sense?<br>
The recommendations have close similarity scores and are from similar genres or time periods.<br>
The recommendations make sense, for example, for user 287 both the content based and collaborative filtering reocmmendations are more highly recommending science fiction movies.